In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time
import pprint
from itertools import combinations
import pickle
import config.config as conf

In [3]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select \
                  x.q_id \
                , x.q_posttypeid \
                , x.q_acceptedanswerid \
                , x.q_parentid \
                , x.q_creationdate \
                , x.q_score \
                , x.q_viewcount \
                , x.q_owneruserid \
                , x.q_title \
                , x.q_tags \
                , x.q_answercount \
                , x.q_commentcount \
                , x.q_favoritecount \
                , x.q_closeddate \
                , b.id               as a_id \
                , b.posttypeid       as a_posttypeid \
                , b.acceptedanswerid as a_acceptedanswerid \
                , b.parentid         as a_parentid \
                , b.creationdate     as a_creationdate \
                , b.score            as a_score \
                , b.viewcount        as a_viewcount \
                , b.owneruserid      as a_owneruserid \
                , b.title            as a_title \
                , b.tags             as a_tags \
                , b.answercount      as a_answercount \
                , b.commentcount     as a_commentcount \
                , b.favoritecount    as a_favoritecount \
                , b.closeddate       as a_closeddate \
  from ( \
           select a.id               as q_id \
                , a.posttypeid       as q_posttypeid \
                , a.acceptedanswerid as q_acceptedanswerid \
                , a.parentid         as q_parentid \
                , a.creationdate     as q_creationdate \
                , a.score            as q_score \
                , a.viewcount        as q_viewcount \
                , a.owneruserid      as q_owneruserid \
                , a.title            as q_title \
                , a.answercount      as q_answercount \
                , a.commentcount     as q_commentcount \
                , a.favoritecount    as q_favoritecount \
                , a.closeddate       as q_closeddate \
                , replace(replace(lower(a.tags), '<', ''), '>', ' ')as q_tags \
           from posts a \
           where a.creationdate between '2013-11-01' and '2023-12-01' \
             and a.posttypeid = '1' \
             and a.owneruserid is not null \
       )   x \
        , posts b \
where b.parentid = x.q_id \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [4]:
df_da = pd.DataFrame(rows, columns = [
  'q_id' 
, 'q_posttypeid' 
, 'q_acceptedanswerid'
, 'q_parentid' 
, 'q_creationdate' 
, 'q_score' 
, 'q_viewcount' 
, 'q_owneruserid' 
, 'q_title' 
, 'q_tags' 
, 'q_answercount' 
, 'q_commentcount' 
, 'q_favoritecount'
, 'q_closeddate' 
, 'a_id' 
, 'a_posttypeid' 
, 'a_acceptedanswerid' 
, 'a_parentid' 
, 'a_creationdate' 
, 'a_score' 
, 'a_viewcount' 
, 'a_owneruserid' 
, 'a_title' 
, 'a_tags' 
, 'a_answercount'
, 'a_commentcount'
, 'a_favoritecount'
, 'a_closeddate' 
])

In [5]:
# 전체 데이터셋 
with open('../../data/df_da.pkl', 'wb') as f:
    pickle.dump(df_da, f)

In [6]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select a.creationdate \
     , a.id \
     , a.posttypeid \
     , a.title \
     , b.body \
     , case when c.id is null then 0 else 1 end as answered_yn \
  from posts as a \
     inner join postsbody b \
     on a.id = b.id \
     left outer join vw_posts_get_answer as c \
     on a.id = c.id \
where a.creationdate between '2013-11-01' and '2023-12-01' \
  ; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [7]:
df_body = pd.DataFrame(rows, columns = [
  'creationdate' 
, 'id' 
, 'posttypeid'
, 'title'
, 'body' 
, 'answered_yn'
])

In [8]:
# 전체 데이터셋 
with open('../../data/df_body.pkl', 'wb') as f:
    pickle.dump(df_body, f)

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select id, creationdate, displayname, lastaccessdate \
  from users \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df_user = pd.DataFrame(rows, columns = [
  'id' 
, 'creationdate' 
, 'displayname'
, 'lastaccessdate'
])

In [ ]:
# 전체 데이터셋 
with open('../../data/df_user.pkl', 'wb') as f:
    pickle.dump(df_user, f)